In [ ]:
# ✅ Step 1: Install Required Libraries
!pip install -q transformers datasets onnx onnxruntime onnxruntime-tools scikit-learn

# ✅ Step 2: Import Libraries
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score

# ✅ Step 3: Load Dataset & Tokenizer
dataset = load_dataset("clinc_oos", "plus")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
num_labels = dataset["train"].features["intent"].num_classes  # should be 150

# ✅ Step 4: Preprocess & Clamp Labels
def preprocess(example):
    encoded = tokenizer(example["text"], padding="max_length", truncation=True, max_length=32)
    label = int(example["intent"])
    encoded["labels"] = label if label < num_labels else num_labels - 1  # clamp safety
    return encoded

dataset = dataset.map(preprocess, batched=False)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ✅ Step 5: Load Model with Correct Output Size
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# ✅ Step 6: TrainingArguments (W&B disabled, warnings handled)
training_args = TrainingArguments(
    output_dir="./bert-intent",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    report_to="none"  # disables wandb
)

# ✅ Step 7: Trainer with Metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 735.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

...

Map:   0%|          | 0/15250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)



<ipython-input-2-a8c0d7bd7cd1>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# ✅ Step 8: Train (on GPU)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.746400,0.317649,0.943548
2,0.097200,0.244481,0.951613
3,0.028500,0.224439,0.957097


TrainOutput(global_step=5721, training_loss=0.6528996319396578, metrics={'train_runtime': 538.4946, 'train_samples_per_second': 84.959, 'train_steps_per_second': 10.624, 'total_flos': 753339653424000.0, 'train_loss': 0.6528996319396578, 'epoch': 3.0})

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
from onnxruntime.quantization import quantize_dynamic, QuantType
from google.colab import files

# ✅ Load best-performing checkpoint
checkpoint_path = "./bert-intent/checkpoint-5721"
model = BertForSequenceClassification.from_pretrained(checkpoint_path)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.eval()

# ✅ Prepare dummy input for ONNX export
inputs = tokenizer("hello world", return_tensors="pt", max_length=32, padding="max_length", truncation=True)

# ✅ Export to FP32 ONNX
torch.onnx.export(
    model,
    (inputs["input_ids"], inputs["attention_mask"]),
    "intent_fp32.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "seq_len"},
        "attention_mask": {0: "batch_size", 1: "seq_len"},
        "logits": {0: "batch_size"}
    },
    opset_version=14
)
print("✅ Exported to intent_fp32.onnx")

# ✅ Quantize to INT8
quantize_dynamic(
    model_input="intent_fp32.onnx",
    model_output="intent_int8.onnx",
    weight_type=QuantType.QInt8
)
print("✅ Quantized to intent_int8.onnx")

# ✅ Download both files
files.download("intent_fp32.onnx")
files.download("intent_int8.onnx")


✅ Exported to intent_fp32.onnx


✅ Quantized to intent_int8.onnx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>